<a href="https://colab.research.google.com/github/dagyaom/Python_Practice/blob/main/%EA%B2%B0%EC%A0%95%ED%8A%B8%EB%A6%AC_%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 결정트리(Decision Trees)

## Merge사용한 데이터 불러오기
- 나누어진 feature와 target을 합한다. 

In [ ]:
import pandas as pd

In [ ]:
train_1 = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv')
train_1.head(3)

In [ ]:
train_2 = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')
train_2.head(3)

In [ ]:
# 데이터 가져오기
target = 'vacc_h1n1_f'
train1 = pd.merge(pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train.csv'),
                 pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/train_labels.csv')[target], left_index=True, right_index=True)
train1.head(3)

# 왼쪽 데이터프레임의 인덱스를 조인키로 사용함.

In [ ]:
test = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/test.csv')
test.head(3)

In [ ]:
sample_submission = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/vacc_flu/submission.csv')
sample_submission.head(3)

## EDA(탐색적 데이터분석)

In [ ]:
# 전체적인 형태 파악
train1.head().T

In [ ]:
# 컬럼 타입 파악
train1.dtypes

### 훈련/검증/테스트 데이터로 나누기

In [ ]:
# 전처리

from sklearn.model_selection import train_test_split

train, val = train_test_split(train1, train_size=0.80, test_size=0.20, 
                              stratify=train1[target], random_state=2)


train.shape, val.shape, test.shape

#stratify : 지정한 Data의 비율을 유지한다. 예를 들어, Label Set인 Y가 25%의 0과 75%의 1로 이루어진 Binary Set일 때,
#  stratify=Y로 설정하면 나누어진 데이터셋들도 0과 1을 각각 25%, 75%로 유지한 채 분할된다.

# 출처: https://ebbnflow.tistory.com/126 [Dev Log : 삶은 확률의 구름]

# ★ stratify: default=None 입니다. classification을 다룰 때 매우 중요한 옵션값입니다. stratify 값을 target으로 지정해주면 각각의 class 비율(ratio)을 train / validation에 유지해 줍니다. 
#(한 쪽에 쏠려서 분배되는 것을 방지합니다) 만약 이 옵션을 지정해 주지 않고 classification 문제를 다룬다면, 성능의 차이가 많이 날 수 있습니다.
# 계층적 데이터 추출 옵션 (분류 모델에서 추천!)
# : 여러 층으로 분할후 각 층별로 렌덤 데이터 추출, 원래 데이터의 분포와 유사하게 데이터 추출
#https://teddylee777.github.io/scikit-learn/train-test-split

In [ ]:
train1[target].value_counts(normalize=True)

In [ ]:
# 타겟 비율 살펴보기
print(train[target].value_counts(normalize=True))
print(val[target].value_counts(normalize=True))

# stratify=train1[target] -- 이 옵션을 줌으로써 train1의 타겟값 비율이 train과 val에 동일하게 적용되었음을 알 수 있다. 
# 클래스가 0,1인 분류문제로, 가장 큰 범주(0)dl 76%가량을 차지하고 있다. --> 클래스가 불균형한 분류문제(imbalanced)

### 프로파일링을 이용한 EDA

In [ ]:
# 프로파일링을 사용해서 데이터리포트를 분석해보자
# pip install pandas-profiling   #이 셀안에 글자적으면 실행안됨.


# out : SyntaxError: invalid syntax

# 설명 : https://github.com/pandas-profiling/pandas-profiling
# 공식문서 : https://pandas-profiling.github.io/pandas-profiling/docs/master/rtd/

In [ ]:
# pip install -U pandas-profiling

In [ ]:
# from pandas_profiling import ProfileReport
# profile = ProfileReport(train).to_notebook_iframe()

# #profile = df.profile_report() 이런 형태도 가능
# # 좀 더 간략하고 빠르게 보고싶다면  ProfileReport(train, minimal=True) 옵션을 지정한다. --> overview와 Variables까지만 나옴.



In [ ]:
# 내보내기
# ProfileReport(train).to_file("your_report.html")

In [ ]:
# warning의 대부분이 결측치에 대한 것임을 알 수 있다. 
# Impute를 사용할 것을 생각해두자.
# dtypes로 하였을 때 실수형이 많았는데, 리포트에는 실수형이 거의 없었다. 어찌된 일인가 알아보자.

## 다시 EDA

In [ ]:
# 실수 형태만 데이터에서 빼내기 : select_dtypes('')
train.select_dtypes('float').head(5).T

# 실수형이지만, 실제로는 범주형이나 이진형 특성들이구나. 

In [ ]:
# 중복값도 있는지 알아보자
train.T.duplicated()

In [ ]:
# 너무 많은 범주 처리:cardinality --> 유일값이 몇개인지 볼 것.
train.describe(exclude='number')

#exclude='' : 결과에서 생략할 데이터 유형의 블랙리스트. 숫자 유형을 제외하려면 numpy.number를 제출하십시오.

In [ ]:
# 디폴트와 비교
train.describe()

In [ ]:
train.describe(exclude='number').T.sort_values(by='unique')

# state 카디널리티가 너무 높아서 조정을 해줘야겠다..!

In [ ]:
# 카테고리를 많이 가지는 특성의 범주도 살펴보자
train['employment_occupation'].value_counts()  #유일값별 갯수세기

## Feature Engneering

In [ ]:
import numpy as np

def engineer(df):
    #높은 카디널리티 제거
    selected_cols = df.select_dtypes(include=['number', 'object'])   # df에서 number와 object유형인 feature를 전부 뽑아낸다.(사실 이 데이터에서는 전부다임)
    labels = selected_cols.nunique()
    selected_features = labels[labels<=30].index.tolist()
    df = df[selected_features]

    #새로운 특성 생성
    #컬럼에서 behavioral 관련 특성 뽑아서 합계한 하나의 특성을 만들기
    behaviorals = [col for col in df.columns if 'behavioral' in col]
    df['behaviorals'] = df[behaviorals].sum(axis=1)

    #seas 컬럼은 제거
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace = True)

    return df

# train = engineer(train)
# val = engineer(val)
# teste = engineer(test)

### 피쳐엔지니어링 연습

In [ ]:
#1) nunique()  vs .unique()
labels1 = train.nunique()  #nunique()는 컬럼별로 유일값 갯수를 세어주는 것.
labels1

In [ ]:
train['employment_occupation'].unique()

# unique()는 어레이배열에서 유일값을 빼주는 것.
# 만약 어레이 배열에서 유일값 갯수를 세고싶다면 value_counts() 메서드를 써야 한다.

In [ ]:
#2) 
labels1[labels1 <=30]  #state가 빠짐.

In [ ]:
select = labels1[labels1 <=30].index # 유일값이 30보다 작은 것만 뽑아서, 어레이 배열의 인덱스를 꺼낸다. 
select

In [ ]:
# 그걸 다시 list로 만든다 ★ 굳이 왜 만든건지 모르겠넹
select1 = labels1[labels1 <=30].index.tolist()
select1

In [ ]:
train_sl=train[select1]
train_sl.head()

In [ ]:
a=[]
for col in train_sl.columns:
    if 'behavioral' in col :
        a.append(col)
    
print(a)

# --> 축약하면
# --> [col for col in train_sl.columns if 'behivioral' in col]

In [ ]:
train_sl[a].head()

In [ ]:
train_sl[a].sum()

In [ ]:
train_sl[a].sum(axis=1)

In [ ]:
train[select].head()

### 연습 끝!

In [ ]:
# 함수 적용해서 특성공학 완료
train = engineer(train)
val = engineer(val)
test = engineer(test)

## 특성/타겟 분리하기

In [ ]:
target = 'vacc_h1n1_f'
features = train.drop(columns = [target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

## Pipelines
- 중복코드의 최소화로 여러 모델을 같은 전처리 프로세스에 연결시킨다.
- 그리드서치(grid search)를 통해 여러 하이퍼파라미터를 쉽게 연결가능.(★??)

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html